# PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.

## Importing the datasets.

In [1]:
# Default libraries: pandas, numpy
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import sqlite3

import math

# Load collections of functions
from matplotlib import pyplot as plt
from collections import Counter
from datetime import datetime

In [2]:
# A. IMPORT DATA
main_path = 'C:\\Users\\map_f\\OneDrive\\Documents\\Dropbox\\Springboard\\Project\\'
project_path = 'Country_Club\\'

path = main_path + project_path

# parser = TextFileReader(fp_or_buf, **kwds)
bookings = pd.read_csv(path + 'country_club_bookings.csv',index_col= None,delimiter=',')
facilities = pd.read_csv(path + 'country_club_facilities.csv',index_col= None,delimiter=',')
members = pd.read_csv(path + 'country_club_members.csv',index_col= None,delimiter=',')

# type(raw_data)

In [3]:
# Create engine: engine
engine = create_engine('sqlite:///country_club.csv')

# help(bookings.to_sql)
bookings.to_sql('bookings',con=engine,if_exists='append')
facilities.to_sql('facilities',con=engine,if_exists='append')
members.to_sql('members',con=engine,if_exists='append')

In [4]:
df = pd.read_sql_query("SELECT * FROM bookings", engine)
# df = pd.read_sql_query("SELECT * FROM facilities", engine)
# df = pd.read_sql_query("SELECT * FROM members", engine)

# Print head of DataFrame
print(df.head())

   index  bookid  facid  memid            starttime  slots
0      0       0      3      1  2012-07-03 11:00:00      2
1      1       1      4      1  2012-07-03 08:00:00      2
2      2       2      6      0  2012-07-03 18:00:00      2
3      3       3      7      1  2012-07-03 19:00:00      2
4      4       4      8      1  2012-07-03 10:00:00      1


### Q10:
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [9]:
df = pd.read_sql_query("SELECT f.name AS facility_name ,SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS revenue FROM Facilities AS f LEFT JOIN Bookings AS b ON b.facid = f.facid GROUP BY f.name HAVING revenue < 1000 ORDER BY revenue", engine)
print(df)

   facility_name  revenue
0   Table Tennis    180.0
1  Snooker Table    240.0
2     Pool Table    270.0


### Q11:
Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [15]:
df = pd.read_sql_query("SELECT DISTINCT m.surname || ', ' || m.firstname AS member_name ,m.recommendedby ,r.surname || ', ' || r.firstname AS recommended_by FROM Members AS m LEFT JOIN Members AS r ON r.memid = m.recommendedby WHERE m.recommendedby > 0 ORDER BY recommended_by, member_name", engine)
print(df)

                 member_name  recommendedby      recommended_by
0          Sarwin, Ramnaresh           15.0     Bader, Florence
1               Coplin, Joan           16.0      Baker, Timothy
2           Genting, Matthew            5.0     Butters, Gerald
3             Baker, Timothy           13.0     Farrell, Jemima
4              Pinker, David           13.0     Farrell, Jemima
5          Rumney, Henrietta           20.0    Genting, Matthew
6             Jones, Douglas           11.0        Jones, David
7                Dare, Nancy            4.0    Joplette, Janice
8               Jones, David            4.0    Joplette, Janice
9                 Hunt, John           30.0  Purview, Millicent
10               Boothe, Tim            3.0         Rownam, Tim
11           Butters, Gerald            1.0       Smith, Darren
12          Joplette, Janice            1.0       Smith, Darren
13           Mackenzie, Anna            1.0       Smith, Darren
14             Owen, Charles            

### Q12:
Find the facilities with their usage by member, but not guests.

In [18]:
df = pd.read_sql_query("SELECT DISTINCT f.name AS facility ,m.surname || ', ' || m.firstname AS member_name ,COUNT(bookid) AS member_usage FROM (SELECT * FROM Bookings AS b WHERE memid != 0) AS b LEFT JOIN Facilities AS f ON f.facid = b.facid LEFT JOIN Members AS m ON m.memid = b.memid GROUP by f.name, m.surname, m.firstname ORDER BY facility, member_name", engine)
print(df)

            facility       member_name  member_usage
0    Badminton Court   Bader, Florence             9
1    Badminton Court       Baker, Anne            10
2    Badminton Court    Baker, Timothy             7
3    Badminton Court       Boothe, Tim            12
4    Badminton Court   Butters, Gerald            20
..               ...               ...           ...
197   Tennis Court 2     Smith, Darren            19
198   Tennis Court 2       Smith, Jack             1
199   Tennis Court 2      Smith, Tracy             2
200   Tennis Court 2  Stibbons, Ponder            31
201   Tennis Court 2     Tracy, Burton             3

[202 rows x 3 columns]


### Q13:
Find the facilities usage by month, but not guests

In [22]:
df = pd.read_sql_query("SELECT DISTINCT f.name AS facility ,strftime('%m', starttime) AS month ,COUNT(bookid) AS count_usage FROM (SELECT * FROM Bookings AS b WHERE memid != 0) AS b LEFT JOIN Facilities AS f ON f.facid = b.facid LEFT JOIN Members AS m ON m.memid = b.memid GROUP by f.name, strftime('%m', starttime) ORDER BY facility, month", engine)
print(df)

           facility month  count_usage
0   Badminton Court    07           51
1   Badminton Court    08          132
2   Badminton Court    09          161
3    Massage Room 1    07           77
4    Massage Room 1    08          153
5    Massage Room 1    09          191
6    Massage Room 2    07            4
7    Massage Room 2    08            9
8    Massage Room 2    09           14
9        Pool Table    07          103
10       Pool Table    08          272
11       Pool Table    09          408
12    Snooker Table    07           68
13    Snooker Table    08          154
14    Snooker Table    09          199
15     Squash Court    07           23
16     Squash Court    08           85
17     Squash Court    09           87
18     Table Tennis    07           48
19     Table Tennis    08          143
20     Table Tennis    09          194
21   Tennis Court 1    07           65
22   Tennis Court 1    08          111
23   Tennis Court 1    09          132
24   Tennis Court 2    07